In [1]:
from dotenv import load_dotenv
# Path to env file containing the waterbodies database credentials
# Only necessary on the Sandbox.
dotenv_path = "/home/jovyan/.env"
load_dotenv(dotenv_path=dotenv_path, verbose=True, override=True)

True

In [2]:
import json
import logging
import sys

import click
from datacube import Datacube
from odc.stats.model import DateTimeRange

from waterbodies.hopper import create_tasks_from_scenes
from waterbodies.io import check_directory_exists, find_geotiff_files
from waterbodies.logs import logging_setup
from waterbodies.text import format_task, get_tile_id_tuple_from_filename

In [3]:
verbose =1
run_type = "gap-filling"
temporal_range = "2024-02--P1M"
historical_extent_rasters_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/historical_extent_rasters/"

In [4]:
# Set up logging.
logging_setup(verbose)
_log = logging.getLogger(__name__)

In [5]:
if not check_directory_exists(path=historical_extent_rasters_directory):
    e = FileNotFoundError(f"Directory {historical_extent_rasters_directory} does not exist!")
    _log.error(e)
    raise e
else:
    historical_extent_rasters = find_geotiff_files(directory_path=historical_extent_rasters_directory)

[2024-04-05 12:20:11,825] {credentials.py:557} INFO - Found credentials in environment variables.


In [6]:
# Get the tile_ids for tiles that actually contain waterbodies.
tile_ids_of_interest = [
        get_tile_id_tuple_from_filename(file_path=raster_file)
        for raster_file in historical_extent_rasters
    ]

In [7]:
product = "wofs_ls"

In [8]:
# Parse the temporal range
temporal_range_ = DateTimeRange(temporal_range)

In [9]:
# Connect to the datacube
dc = Datacube(app="gap-filling")

In [10]:
# The difference between gap-filling and the other steps is here
# we are searching for datasets by their creation date (`creation_time`),
# not their acquisition date (`time`).
dc_query = dict(product=product, creation_time=(temporal_range_.start, temporal_range_.end))
dc_query

{'product': 'wofs_ls',
 'creation_time': (datetime.datetime(2024, 2, 1, 0, 0),
  datetime.datetime(2024, 2, 29, 23, 59, 59, 999999))}

In [11]:
%%time
# Search the datacube for all wofs_ls datasets whose creation times (not acquisition time)
# fall within the temporal range specified.
# E.g  a dataset can have an aquisition date of 2023-12-15 but have been added to the
# datacube in 2024-02, which will be its creation date.
scenes = dc.find_datasets(**dc_query)

CPU times: user 2.14 s, sys: 298 ms, total: 2.44 s
Wall time: 1min 12s


In [12]:
%%time
# Get the ids of the tasks to process.
tasks_ = create_tasks_from_scenes(scenes=scenes, tile_ids_of_interest=tile_ids_of_interest)
task_ids = [task_id for task in tasks_ for task_id, task_dataset_ids in task.items()]

Processing   21,786 scenes: 100%|██████████| 21786/21786 [00:17<00:00, 1233.23it/s]

[2024-04-05 12:21:42,957] {hopper.py:167} INFO - Filter the 5090 cells to keep only the cells containing the 2783 tile ids of interest.


[2024-04-05 12:21:43,121] {hopper.py:174} INFO - Total number of cells after filtering: 2783
[2024-04-05 12:21:43,122] {hopper.py:178} INFO - For each cell, group the datasets by solar day
[2024-04-05 12:21:44,096] {hopper.py:184} INFO - Total number of tasks: 85014
CPU times: user 18.8 s, sys: 65.7 ms, total: 18.9 s
Wall time: 18.8 s


In [13]:
# For each task id add an empty list as a place holder for the task's datasets' ids
# which will be filled  during processing. This is because the processing step is expected
# to be done in parallel hence filling the task's datasets' ids will be faster there than
# looping over each task to update the required datasets' ids here.
tasks = [{task_id: []} for task_id in task_ids]

In [14]:
# Put the tasks in the correct format.
tasks = [format_task(task) for task in tasks]

In [15]:
json.dump(tasks[:10], sys.stdout)

[{"solar_day": "2024-01-11", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}, {"solar_day": "2024-01-27", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}, {"solar_day": "2024-02-03", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}, {"solar_day": "2024-02-04", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}, {"solar_day": "2024-01-16", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}, {"solar_day": "2024-02-11", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}, {"solar_day": "2024-01-26", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}, {"solar_day": "2024-01-19", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}, {"solar_day": "2024-01-18", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}, {"solar_day": "2023-09-25", "tile_id_x": 199, "tile_id_y": 66, "task_datasets_ids": []}]